#### 01 환경설정

In [4]:
!pip install --quiet langchain openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.4 MB/s eta 0:00:00


In [5]:
# 사용자 환경 설정
# modulabs통해 발급받은 API-KEY 사용
# 추후 open-ai, gemini KEY 발급시에도 아래처럼 환경 설정 가능

# 01 Colab용 비밀 키 불러오기
from google.colab import userdata
import os
from openai import OpenAI  # openai-python SDK

# 02 OpenAI API 키 설정
api_key = userdata.get("modulabsKey")  # Colab 사용자 환경변수 탭에서 등록한 키
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI(api_key=api_key)

# 03 버전 확인 (선택)
import openai
import langchain
print("LangChain ver:", langchain.__version__)
print("OpenAI SDK ver:", openai.__version__)

LangChain ver: 0.3.25
OpenAI SDK ver: 1.86.0


#### 02 output-parser 없는 형태

In [6]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

llm = ChatOpenAI(model_name="gpt-4o-mini",
                 temperature = 0)

#ChatPromptTemplate에 SystemMessage로 LLM의 역할과 출력 형식 지정
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 영화 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘."
                'ex) Query: SF영화 3개 추천해줘 / 답변: ["인터스텔라", "스페이스오디세이", "혹성탈출"]'
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="스릴러 영화 3개를 추천해줘.")
answer = llm.invoke(messages)
result = answer.content
print(result)

["세븐", "파이트 클럽", "조커"]


#### 03 CSV파서

In [7]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# CSV파서 선언
output_parser = CommaSeparatedListOutputParser()
# CSV파서 작동을 위한 형식 지정 프롬프트 코드
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿의 partial_variables에 CSV형식 지정 프롬프트 주입

prompt = PromptTemplate(
    template = "List {number} {subject}. answer in Korean. \n{format_instructions}",
    input_variables = ["subject","number"],
    partial_variables = {"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)

# 프롬프트 템플릿-모델-Output Parser를 체인으로 연결
chain = prompt | model | output_parser
chain.invoke({"subject":"공포 영화","number":"4"})

['곤지암', '겟 아웃', '살인의 추억', '실미도']

In [8]:
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

#### 04 Datetime 파서
- 날짜 형식만 출력하는 Datetime 파서
- Datetime 파서는 LLM의 답변을 파이썬의 Datetime객체 형식으로 출력하도록 함

In [9]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

output_parser = DatetimeOutputParser()

template = """
    Answer the users question:
    {question}

    {format_instructions}
"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

model = ChatOpenAI(temperature=0)

chain = prompt | model | output_parser
output = chain.invoke({"question": "비트코인은 언제 개발됐어?"})
print(output)

2009-01-03 18:15:05


In [10]:
output_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1270-01-25T12:06:14.168777Z, 638-04-08T00:14:02.358086Z, 1467-07-19T09:29:09.789726Z\n\nReturn ONLY this string, no other words!"

#### 05 JSON파서
- 자바스크립트 클라이언트와 백엔드 서버 간의 통신을 위한 형식으로 개발됨
- 대부분의 웹 사이트가 자바스크립트에 기반하여 사용자와 서버 간의 통신한다는 점 고려하면,   
개발자 친화적인 형식임

In [11]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define your desired data structure
class Country(BaseModel):
  continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
  population: str = Field(description="사용자가 물어본 나라의 인구(int 형식)")
# And a query intended to prompt a language model to populate the data structure
country_query = "아르헨티나는 어떤 나라야?"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Country)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
chain.invoke({"query": country_query})

{'continent': '남아메리카', 'population': '45000000'}